In [1]:
import numpy as np
import re
import json
import pandas as pd

In [2]:
model = "text-ada-001"

labels = pd.read_csv("results.txt", delimiter = ",").loc[:,["pairid", " varA","varB","dataset","groundtruth"]]
labels['pairid'] = labels['pairid'].str.extract('(\d{4})').astype(int) # convert pairid to int
labels["groundtruth"] = labels["groundtruth"].str.replace(r'\s+', '', regex=True) # remove white space in labels

weights = pd.read_csv("pairmeta.txt", delimiter = " ", header = None).iloc[:,[0,5]]
weights.rename(columns = {0:"pairid", 5:"weight"}, inplace=True)
weights['pairid'] = weights['pairid'].astype(int) # convert pairid to int

merged_df = labels.merge(weights, on='pairid')
merged_df["AtoB"] = ""
merged_df["BtoA"] = ""

with open(model + "_response.json", "r") as file:
    loaded_data = json.load(file)
    
merged_df.head()

,pairid,varA,varB,dataset,groundtruth,weight,AtoB,BtoA
0,1,Altitude,Temperature,DWD,->,0.166,,
1,2,Altitude,Precipitation,DWD,->,0.166,,
2,3,Longitude,Temperature,DWD,->,0.167,,
3,4,Altitude,Sunshine hours,DWD,->,0.166,,
4,5,Age,Length,Abalone,->,0.143,,


In [3]:
for i in range(108):
    print(loaded_data[i])

{'pair0001,_res1': {'warning': 'This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations', 'id': 'cmpl-7z65kn4NP00BRBfLe83M2ThTsxgqU', 'object': 'text_completion', 'created': 1694795780, 'model': 'text-ada-001', 'choices': [{'text': '\n\nYes', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 22, 'completion_tokens': 3, 'total_tokens': 25}}}
{'pair0001,_res2': {'warning': 'This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations', 'id': 'cmpl-7z65kxYh0Dmamrhk5AJsP8gAZvMh2', 'object': 'text_completion', 'created': 1694795780, 'model': 'text-ada-001', 'choices': [{'text': '\n\nYes', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 20, 'completion_tokens': 3, 'total_tokens': 23}}}
{'pair0002,_res1': {'warning': 'This model version

In [12]:
def add_prediction(merged_df, loaded_data, model):
    merged_df = merged_df.set_index('pairid')
    for response in loaded_data:
        pairid, resid, answer = extract_answers(response)
        print(pairid,resid)
        col = "AtoB" if resid == "1" else "BtoA"
        merged_df.loc[int(pairid),col] = answer if answer[:3] == "Yes" else "No"
        print("Converted:", merged_df.loc[int(pairid),col])
    return merged_df


def extract_answers(s):
    # Compile regex patterns
    
    pairid_pattern = re.compile(r"\{'pair0(\d{3}),_res(\d)")
    
    # Search for pairid
    pairid_match = pairid_pattern.search(str(s))
    pairid = pairid_match.group(1) if pairid_match else None
    resid = pairid_match.group(2) if pairid_match else None
    
    # Search for answer
    ans = s["pair0"+ str(pairid) + ',_res' + str(resid)]["choices"][0]["text"]
    
    return pairid, resid, ans[2:]

In [13]:
results = add_prediction(merged_df, loaded_data, model)

001 1
Converted: Yes
001 2
Converted: Yes
002 1
Converted: Yes
002 2
Converted: Yes
003 1
Converted: Yes
003 2
Converted: Yes
004 1
Converted: Yes
004 2
Converted: Yes
005 1
Converted: Yes
005 2
Converted: Yes
006 1
Converted: Yes
006 2
Converted: Yes
007 1
Converted: Yes
007 2
Converted: Yes
008 1
Converted: Yes
008 2
Converted: Yes
009 1
Converted: Yes
009 2
Converted: Yes
010 1
Converted: Yes
010 2
Converted: Yes
011 1
Converted: Yes
011 2
Converted: Yes
012 1
Converted: Yes
012 2
Converted: Yes
013 1
Converted: Yes
013 2
Converted: Yes
014 1
Converted: Yes
014 2
Converted: Yes
015 1
Converted: Yes
015 2
Converted: Yes
016 1
Converted: Yes
016 2
Converted: Yes
017 1
Converted: Yes
017 2
Converted: Yes
018 1
Converted: Yes
018 2
Converted: Yes
019 1
Converted: Yes
019 2
Converted: Yes
020 1
Converted: Yes
020 2
Converted: Yes
021 1
Converted: Yes
021 2
Converted: Yes
022 1
Converted: Yes
022 2
Converted: Yes
023 1
Converted: Yes
023 2
Converted: Yes
024 1
Converted: Yes
024 2
Convert

In [14]:
results

,varA,varB,dataset,groundtruth,weight,AtoB,BtoA
pairid,,,,,,,
1,Altitude,Temperature,DWD,->,0.166,Yes,Yes
2,Altitude,Precipitation,DWD,->,0.166,Yes,Yes
3,Longitude,Temperature,DWD,->,0.167,Yes,Yes
4,Altitude,Sunshine hours,DWD,->,0.166,Yes,Yes
5,Age,Length,Abalone,->,0.143,Yes,Yes
...,...,...,...,...,...,...,...
104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->,0.200,Yes,Yes
105,pixel vector of a patch,total brightness at the screen,D. Janzing,->,0.500,Yes,Yes
106,time required for one round,voltage,D. Janzing,<-,1.000,Yes,Yes


In [15]:
results["gt_AtoB"] = np.where(results["groundtruth"] == "->", "Yes", "No")
results["gt_BtoA"] = np.where(results["groundtruth"] == "->", "No", "Yes")
results["correct_AtoB"] = np.where(results["gt_AtoB"] == results["AtoB"], 1, 0)
results["correct_BtoA"] = np.where(results["gt_BtoA"] == results["BtoA"], 1, 0)
wtd_acc = (sum(results["correct_AtoB"]*results["weight"]) + sum(results["correct_BtoA"]*results["weight"])) / (results["weight"].sum()*2)
acc = (sum(results["correct_AtoB"]) + sum(results["correct_BtoA"])) / 108 / 2
print("acc:", acc, "weighted acc:", wtd_acc)

acc: 0.5092592592592593 weighted acc: 0.5085478961687682


In [17]:
results["correct"] = np.where((results["BtoA"] == results["gt_BtoA"]) & (results["gt_AtoB"] == results["AtoB"]), 1, 0)
wtd_acc = (sum(results["correct"]*results["weight"])) / (results["weight"].sum())
acc = (sum(results["correct"])) / 108
print("if we count both correct as correct: ")
print("acc:", acc, "weighted acc:", wtd_acc)

if we count both correct as correct: 
acc: 0.1111111111111111 weighted acc: 0.08087614974139633


In [9]:
results

,varA,varB,dataset,groundtruth,weight,AtoB,BtoA,gt_AtoB,gt_BtoA,correct_AtoB,correct_BtoA,correct
pairid,,,,,,,,,,,,
1,Altitude,Temperature,DWD,->,0.166,Yes,Yes,Yes,No,1,0,0
2,Altitude,Precipitation,DWD,->,0.166,Yes,Yes,Yes,No,1,0,0
3,Longitude,Temperature,DWD,->,0.167,Yes,Yes,Yes,No,1,0,0
4,Altitude,Sunshine hours,DWD,->,0.166,Yes,Yes,Yes,No,1,0,0
5,Age,Length,Abalone,->,0.143,Yes,Yes,Yes,No,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->,0.200,Yes,Yes,Yes,No,1,0,0
105,pixel vector of a patch,total brightness at the screen,D. Janzing,->,0.500,Yes,Yes,Yes,No,1,0,0
106,time required for one round,voltage,D. Janzing,<-,1.000,Yes,Yes,No,Yes,0,1,0
